In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime

In [2]:
# Read the data file
df=pd.read_csv('LoanStats3b_securev1.csv',header=1)

# Choose some meaningful and available features for predictions
df=df[['loan_amnt','term','int_rate','installment','sub_grade','emp_title','emp_length','home_ownership','annual_inc','verification_status',
        'issue_d','desc','purpose','title','zip_code','addr_state','dti','delinq_2yrs','earliest_cr_line','fico_range_low','fico_range_high','inq_last_6mths',
        'open_acc','revol_bal','revol_util','total_acc','acc_open_past_24mths','mort_acc','percent_bc_gt_75','loan_status']]

# Drop rows that are completely empty
df.dropna(how='all', inplace=True)

# Change the format of int_rate to float
df['int_rate']=df['int_rate'].apply(lambda x: float(x[:-1]))

# Change the format of revol_util to float
def process_revol_util(x):
    if type(x) == str:
        x=float(x[:-1])
    return x
df['revol_util']=df['revol_util'].apply(lambda x: process_revol_util(x))

# Fill in missing values with the mean
df['revol_util'].fillna(value=df['revol_util'].mean(), inplace=True)
df['acc_open_past_24mths'].fillna(value=df['acc_open_past_24mths'].mean(), inplace=True)
df['mort_acc'].fillna(value=df['mort_acc'].mean(), inplace=True)
df['percent_bc_gt_75'].fillna(value=df['percent_bc_gt_75'].mean(), inplace=True)

# Change emp_length and sub_grade from strings to integers
mapping_dict={"emp_length": {"10+ years": 10, "9 years": 9, "8 years": 8, "7 years": 7, "6 years": 6, "5 years": 5, "4 years": 4, 
                               "3 years": 3, "2 years": 2, "1 year": 1, "< 1 year": 0}, 
                "sub_grade":{"G5": 0, "G4": 1, "G3": 2, "G2": 3, "G1": 4, "F5": 5, "F4": 6, "F3": 7, "F2": 8, "F1": 9, "E5": 10,
                             "E4": 11, "E3": 12, "E2": 13, "E1": 14, "D5": 15, "D4": 16, "D3": 17, "D2": 18, "D1": 19, "C5": 20, 
                             "C4": 21, "C3": 22, "C2": 23, "C1": 24, "B5": 25, "B4": 26, "B3": 27, "B2": 28, "B1": 29, "A5": 30, 
                             "A4": 31, "A3": 32, "A2": 33, "A1": 34}}
df=df.replace(mapping_dict)

# Fill in missing values of emp_length based on emp_title
for i in df.index:
    if np.isnan(df.loc[i,'emp_length']):
        title=df.loc[i,'emp_title']
        if type(title)==str:
            average=df[df['emp_title']==title]['emp_length'].mean()
            df.loc[i,'emp_length']=average

# Drop the rows with NaN in either emp_title or emp_length
df.dropna(subset=['emp_length'], inplace=True)

# Change issue_d to datetime format
df['issue_d']=df['issue_d'].apply(lambda x: datetime.datetime.strptime(x, '%b-%Y'))

# Change earliest_cr_line to datetime format
df['earliest_cr_line']=df['earliest_cr_line'].apply(lambda x: datetime.datetime.strptime(x, '%b-%Y'))

# Calculate the differences in months from earliest_cr_line to issue_d
def diff_month(d1,d2):
    return (d1.year-d2.year)*12+(d1.month-d2.month)

for i in df.index:
    df.loc[i,'earliest_cr_line']=diff_month(df.loc[i,'issue_d'],df.loc[i,'earliest_cr_line'])

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Change the target variable to numbers: 1 indicates defaulted, 0 indicates paid off
def flag(x):
    if x=='Fully Paid':
        return 0
    elif x=='Charged Off':
        return 1

df['loan_status']=df['loan_status'].apply(flag)

# Drop loans with other unknown status
df.dropna(subset=['loan_status'], inplace=True)

In [5]:
# Reset the index to integers starting from 0
df.reset_index(drop=True, inplace=True)

In [6]:
df_default=df[df['loan_status']==1]


In [33]:
term_dic={}
home_dic={}
verification_dic={}
purpose_dic={}
state_dic={}


In [18]:
term_df=df.groupby('term').count()
default_term_df=df_default.groupby('term').count()

In [34]:
for i in range(len(term_df.index)):
    term_dic[list(term_df.index)[i]]=default_term_df.iloc[i,0]/term_df.iloc[i,0]

In [35]:
term_dic

{' 36 months': 0.12443232762076933, ' 60 months': 0.2559940639057645}

In [42]:
home_df=df.groupby('home_ownership').count()
default_home_df=df_default.groupby('home_ownership').count()

In [45]:
for i in range(len(home_df.index)):
    home_dic[list(home_df.index)[i]]=default_home_df.iloc[i,0]/home_df.iloc[i,0]

In [46]:
home_dic

{'MORTGAGE': 0.14214771435313298,
 'NONE': 0.14634146341463414,
 'OTHER': 0.20930232558139536,
 'OWN': 0.15888841747984728,
 'RENT': 0.17289187222885696}

In [48]:
veri_df=df.groupby('verification_status').count()
default_veri_df=df_default.groupby('verification_status').count()

In [50]:
for i in range(len(veri_df.index)):
    verification_dic[list(veri_df.index)[i]]=default_veri_df.iloc[i,0]/veri_df.iloc[i,0]

In [51]:
verification_dic

{'Not Verified': 0.12227387707663909,
 'Source Verified': 0.16098672478700218,
 'Verified': 0.17753761129874118}

In [52]:
purpose_df=df.groupby('purpose').count()
default_purpose_df=df_default.groupby('purpose').count()

In [53]:
for i in range(len(purpose_df.index)):
    purpose_dic[list(purpose_df.index)[i]]=default_purpose_df.iloc[i,0]/purpose_df.iloc[i,0]

In [54]:
purpose_dic

{'car': 0.10932475884244373,
 'credit_card': 0.12864160303487185,
 'debt_consolidation': 0.16502194515110005,
 'home_improvement': 0.1346825559893158,
 'house': 0.15834932821497122,
 'major_purchase': 0.12453478385342112,
 'medical': 0.16934919524142758,
 'moving': 0.16243654822335024,
 'other': 0.18883965992096755,
 'renewable_energy': 0.17543859649122806,
 'small_business': 0.2547344968436688,
 'vacation': 0.15690866510538642,
 'wedding': 0.13480578827113482}

In [73]:
state_df=df.groupby('addr_state').count()
default_state_df=df_default.groupby('addr_state').count()
state_df.drop(labels='ID',axis=0, inplace=True)

In [82]:
for i in range(len(state_df.index)):
    state_dic[list(state_df.index)[i]]=default_state_df.iloc[i,0]/state_df.iloc[i,0]

In [89]:
state_dic['ID']=0.0

In [90]:
state_dic

{'AK': 0.1336996336996337,
 'AL': 0.18304779080252478,
 'AR': 0.17559523809523808,
 'AZ': 0.15773809523809523,
 'CA': 0.1480254260210982,
 'CO': 0.12696747114375656,
 'CT': 0.15096359743040685,
 'DC': 0.09057971014492754,
 'DE': 0.16258351893095768,
 'FL': 0.17410237449675459,
 'GA': 0.1427809388335704,
 'HI': 0.17318435754189945,
 'IA': 1.0,
 'IL': 0.13741651271848798,
 'IN': 0.17452402538531278,
 'KS': 0.1373117033603708,
 'KY': 0.15985130111524162,
 'LA': 0.1701931922723091,
 'MA': 0.15253445456669001,
 'MD': 0.1622668927100993,
 'MI': 0.16529701807936134,
 'MN': 0.14477468839884947,
 'MO': 0.1594048884165781,
 'MS': 0.3333333333333333,
 'MT': 0.12226277372262774,
 'NC': 0.1670871337085193,
 'NE': 0.5,
 'NH': 0.11431870669745958,
 'NJ': 0.17451205510907003,
 'NM': 0.16279069767441862,
 'NV': 0.18383761011106856,
 'NY': 0.17130731142691788,
 'OH': 0.16223925833481903,
 'OK': 0.17510944340212634,
 'OR': 0.13240131578947367,
 'PA': 0.16043920026220912,
 'RI': 0.16020671834625322,
 'SC'

In [91]:
df.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc',
       'revol_bal', 'revol_util', 'total_acc', 'acc_open_past_24mths',
       'mort_acc', 'percent_bc_gt_75', 'loan_status'],
      dtype='object')

In [92]:

mapping_dict={"term": term_dic, "home_ownership": home_dic, "verification_status": verification_dic, "purpose": purpose_dic, "addr_state": state_dic}
df=df.replace(mapping_dict)

In [106]:
df.to_csv('2012-2013_cleaned2.csv')